# Laptop Price Prediction - Data Science Notebook

This notebook contains the exploratory data analysis, data preprocessing, and model building for laptop price prediction.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [ ]:
# Load the dataset
df = pd.read_csv('laptop_price.csv', encoding='ISO-8859-1')
df.head()

## Exploratory Data Analysis

In [ ]:
# Distribution of laptop prices
sns.histplot(df['Price_euros'])
plt.title("Distribución de los precios de las laptops")
plt.xlabel("Precio (euros)")
plt.ylabel("Frecuencia")
plt.show()

## Data Preprocessing

In [ ]:
# Clean and transform data
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)
df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)
df = pd.get_dummies(df, columns=['TypeName'], dtype='int')
df[['screen_width', 'screen_height']] = df['ScreenResolution'].str.extract(r'(\d{3,4})x(\d{3,4})').astype(int)
df['GHz']= df['Cpu'].str.split().str[-1].str.replace('GHz', '').astype(float)

# Show transformed data
df.head()

In [ ]:
# Create correlation matrix
plt.figure(figsize=(17,12))
sns.heatmap(df.corr(numeric_only=True), cmap='coolwarm', annot=True)
plt.show()

## Feature Selection and Model Building

In [ ]:
# Select features based on correlation
X = df[['Weight', 'TypeName_Gaming', 'TypeName_Notebook', 'screen_width', 'screen_height', 'GHz', 'Ram']]
y = df['Price_euros']

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
# Linear Regression Model
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# Evaluate model
from sklearn.metrics import mean_absolute_error, r2_score

print("MAE:", mean_absolute_error(y_test, y_pred_lr))
print("R2:", r2_score(y_test, y_pred_lr))

In [ ]:
# Random Forest Model
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest MAE:", mean_absolute_error(y_test, y_pred_rf))
print("Random Forest R2:", r2_score(y_test, y_pred_rf))

## Model Deployment with Gradio

In [ ]:
# Create prediction function
def predecir_precio(weight, typename_gaming, typename_notebook, screen_width, screen_height, ghz, ram):
  # Crear los datos de entrada al modelo
  entrada = np.array([[weight, typename_gaming, typename_notebook, screen_width, screen_height, ghz, ram]])

  # Realizar la predicción
  prediccion = lr.predict(entrada)

  # Devolver el resultado
  return f"Precio estimado:  € {prediccion[0]:,.2f}"

# Create interface
import gradio as gr

interfaz = gr.Interface(
    fn=predecir_precio,
    inputs=[
        gr.Number(label="Peso (Kg)"),
        gr.Number(label="¿Es Gaming? (1 = Sí, 0 = No)"),
        gr.Number(label="¿Es Notebook? (1 = Sí, 0 = No)"),
        gr.Number(label="Ancho de Pantalla (px)"),
        gr.Number(label="Alto de Pantalla (px)"),
        gr.Number(label="GHz del procesador"),
        gr.Number(label="RAM (GB)"),
    ],
    outputs=gr.Textbox(label="Precio estimado (€)")
)

# Launch interface
interfaz.launch()